In [ ]:
# ✅ 전체 코드: LLM + 뉴스 API + DB 라우팅 LangGraph 예제

import os
import requests
from urllib.parse import quote
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph
from dotenv import load_dotenv

load_dotenv()

# ✅ 상태 정의
class QAState(TypedDict):
    question: str
    route_label: str
    retrieved: str
    answer: str

# ✅ LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))

In [14]:
# ✅ 라우팅 판단 노드
def llm_router_update(state: QAState) -> QAState:
    prompt = f"""
    사용자의 질문을 보고 아래 중 어떤 유형인지 판단하세요:

    - general: 일반 지식이나 개념
    - web: 최근 뉴스, 최신 정보
    - db: 회의록, 내부 데이터 요약

    질문: \"{state['question']}\"

    위 3가지 중 하나의 단어(general/web/db)만 반환하세요.
    """
    result = llm.invoke(prompt).content.strip().lower()
    state["route_label"] = result
    print(f"🔀 [LLM 라우팅 판단] → {result}")
    return state

# ✅ 조건 분기 기준
def get_route(state: QAState) -> str:
    return state["route_label"]

In [15]:
# ✅ 일반 응답 노드
prompt_general = ChatPromptTemplate.from_template("""
'{question}'라는 질문에 대해 짧고 간결하게 설명해주세요.
""")

def general_node(state: QAState) -> QAState:
    chain = prompt_general | llm
    response = chain.invoke({"question": state["question"]})
    state["answer"] = response.content.strip()
    return state

# ✅ 뉴스 검색 API 함수
def search_naver_news(keyword: str, display: int = 3):
    url = f"https://openapi.naver.com/v1/search/news.json?query={quote(keyword)}&display={display}&sort=date"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        items = res.json().get("items", [])
        return [f"- {item['title'].replace('<b>', '').replace('</b>', '')}: {item['link']}" for item in items]
    return ["뉴스 검색 실패"]

# ✅ web 노드: 뉴스 API → 요약
def web_node(state: QAState) -> QAState:
    news_list = search_naver_news(state["question"])
    state["retrieved"] = "\n".join(news_list)
    return state

# ✅ db 노드: (현재는 Mock 텍스트)
def db_node(state: QAState) -> QAState:
    state["retrieved"] = "[DB 검색 결과] 회의 내용을 요약했습니다."
    return state

# ✅ generate 노드: retrieved 기반 요약 생성
prompt_summary = ChatPromptTemplate.from_template("""
다음은 문서 또는 뉴스 목록입니다. 핵심적인 내용을 2~3문장으로 요약해 주세요:

{retrieved}
""")

def generate_node(state: QAState) -> QAState:
    if state.get("retrieved"):
        chain = prompt_summary | llm
        response = chain.invoke({"retrieved": state["retrieved"]})
        state["answer"] = response.content.strip()
    return state

In [16]:
# ✅ LangGraph 구성

def build_graph():
    builder = StateGraph(QAState)

    builder.add_node("judge_route", RunnableLambda(llm_router_update))
    builder.add_node("general", RunnableLambda(general_node))
    builder.add_node("web", RunnableLambda(web_node))
    builder.add_node("db", RunnableLambda(db_node))
    builder.add_node("generate", RunnableLambda(generate_node))

    builder.set_entry_point("judge_route")
    builder.add_conditional_edges("judge_route", get_route, {
        "general": "general",
        "web": "web",
        "db": "db"
    })

    builder.add_edge("general", "generate")
    builder.add_edge("web", "generate")
    builder.add_edge("db", "generate")
    builder.set_finish_point("generate")

    return builder.compile()

In [19]:
# ✅ 실행 테스트
if __name__ == "__main__":
    graph = build_graph()
    questions = [
        "AI Agent란 무엇인가요?",
        "오늘의 주요 뉴스 알려줘",
        "지난 회의 요약해줘"
    ]
    for q in questions:
        print(f"\n❓ 질문: {q}")
        result = graph.invoke({"question": q, "route_label": "", "retrieved": "", "answer": ""})
        print(f"✅ 응답: {result['answer']}")


❓ 질문: AI Agent란 무엇인가요?
🔀 [LLM 라우팅 판단] → general
✅ 응답: AI Agent란 특정 작업을 수행하거나 문제를 해결하기 위해 설계된 인공지능 시스템입니다. 이들은 환경을 인식하고, 의사 결정을 내리며, 주어진 목표를 달성하기 위해 행동합니다. AI Agent는 자율적으로 작동할 수 있으며, 다양한 분야에서 활용됩니다.

❓ 질문: 오늘의 주요 뉴스 알려줘
🔀 [LLM 라우팅 판단] → web
✅ 응답: 뉴스 검색에 실패했다는 내용으로, 특정 뉴스나 문서에 대한 정보가 제공되지 않거나 접근할 수 없는 상황을 나타냅니다. 이로 인해 원하는 정보를 찾지 못한 상태입니다.

❓ 질문: 지난 회의 요약해줘
🔀 [LLM 라우팅 판단] → db
✅ 응답: 회의에서는 주요 안건과 의사결정 사항이 논의되었으며, 참석자들은 각자의 의견을 공유하고 향후 계획에 대해 합의했습니다. 또한, 특정 문제에 대한 해결 방안도 제시되어 실질적인 진행 방향이 설정되었습니다.
